# TensorBoard的使用 笔记
教程视频链接：https://www.bilibili.com/video/BV1hE411t7RN

这篇笔记对应视频合集中的
- P7.TensorBoard的使用（一）
- P8.TensorBoard的使用（二）

TensorBoard是pytorch1.1版本添加的新功能，可以显示图片，绘制图像，可以有效的检测模型训练过程中各个阶段的输出情况。

## 1.SummaryWriter使用

In [2]:
#导入TensorBoard(SummaryWriter类)
from torch.utils.tensorboard import SummaryWriter
#该类向log_dir文件夹写入事件文件（event files），写入的内容可以被TensorBoard解析
'''
这里遇到了No Class named SummaryWriter错误，环境中已经安装了pytorch库
重新在conda中运行conda install pytorch tensorboard -c pytorch解决了这个错误
'''

'\n这里遇到了No Class named SummaryWriter错误，环境中已经安装了pytorch库\n重新在conda中运行conda install pytorch tensorboard -c pytorch解决了这个错误\n'

In [25]:
'''
实例化SummaryWriter对象常用初始化参数：
log_dir -> 事件文件存储路径，一般为相对路径，文件夹不存在会自动创建
'''
writer = SummaryWriter(log_dir='logs')

'''
writer.add_image()类用于显示图像。常用参数：
tag -> 图像标题
img_tensor -> 图片。支持的数据类型：tensor、numpy array、string
'''

#例：用add_image显示图像
from PIL import Image
import torchvision

img = Image.open(fp=r'myDataset\train\ants_image\0013035.jpg')
img2 = Image.open(fp=r'myDataset\train\bees_image\16838648_415acd9e3f.jpg')

#用transforms将图像转化为tensor
trans_totensor = torchvision.transforms.ToTensor()
img_tensor = trans_totensor(img)
writer.add_image(tag='an_ant', img_tensor=img_tensor)

#或者用numpy将图像转换为numpy array
import numpy
img_array = numpy.array(img2)
print(img_array.shape)#图片格式是HWC而默认的是CHW(channel height width)
writer.add_image(tag='a_bee', img_tensor=img_array, dataformats='HWC')





'''
writer.add_scalar类用于绘制图表。常用参数：
tag -> 图表标题
global_step -> x坐标
scalar_value -> y坐标
'''

#例：用add_scalar绘制y=x图像
for i in range(100):
    writer.add_scalar(tag='y=2x', global_step=i, scalar_value=2*i)

writer.close()#关闭

(450, 500, 3)


这里用到的PIL在pillow库中，还需要opencv，需要先在终端运行
```bash
conda install pillow
conda install opencv
```

## 2.查看TensorBoard
在终端输入命令，logdir是事件文件存储的路径
```bash
tensorboard --logdir=logs
```
新版tensorboard可以运行
```bash
tensorboard --logdir logs
```
如果报错`Unable to create process`可以运行
```bash
python -m tensorboard.main --logdir=.\logs
```
这里的`-m`表示以运行脚本的方式运行tensorboard的模块

该命令的默认端口是6006如果想要手动设置端口，可以使用
```bash
python -m tensorboard.main --logdir=.\logs --port=6007
```
也可以让程序自行寻找空闲端口
```bash
python -m tensorboard.main --logdir=.\logs --port=0
```

命令成功后，会得到一个链接
```
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.0 at http://localhost:6006/
```
浏览器访问这个链接`http://localhost:6006/`就可以查看tensorboard了

### IDE终端无法运行conda环境的解决方案
#### 错误
我使用的是vscode的python插件，运行`tensorboard --logdir logs`报错`CommandNotFoundException`

在ide里面直接打开终端，提示词没有`(pytorch)`字样，而是`PS E:\path\to\dir>`。微软官方说尽管没有这个提示，实际上conda环境已经启动了（https://github.com/microsoft/vscode-python/wiki/Activate-Environments-in-Terminal-Using-Environment-Variables ）。

然而即便如此，因为我没有把conda添加到PATH，也没有办法通过`conad activate`命令切换到`(base)`以外的conda环境。
#### 解决方案
参考：https://github.com/microsoft/vscode-python/issues/20473

找到`conda activate`对应的.bat文件，在vscode里打开一个cmd终端（用powershell不行）输入`D:\"Program Files"\anaconda\Scripts\activate.bat activate pytorch`，就可以看见`(pytorch)`提示词了。

根据视频弹幕。如果使用pycharm，可以通过设置 -> 终端 -> 路径从shell改为cmd 解决。


### tensorboard和numpy版本兼容问题解决方案
#### 错误
终端里运行``tensorboard`命令报错`AttributeError: `np.string_` was removed in the NumPy 2.0 release. Use `np.bytes_` instead.``，是出现了pytorch中集成的tensorboard和numpy之间兼容性的问题（https://stackoverflow.com/questions/78721195/attributeerror-np-string-was-removed-in-the-numpy-2-0-release-use-np-bytes ）
### 解决方案
tensorflow中集成的tensorboard将兼容性问题处理的更好。可以安装tensorflow和pytorch共同支持的python版本（如3.6），并安装tensorflow包
```bash
conda install tensorflow
```
再启动tensorboard就没问题了

参考：https://blog.51cto.com/u_15521344/5056937

### 其他可能出现的问题和解决方案
1. 有时tensorboard无法写入也无法启动，可以尝试卸载vscode自带的tensorboard插件
2. 仔细检查logs文件夹的相对路径是否正确，很容易写错